In [ ]:
"""Gold Prediction SubModel Training - real_rate Attempt 1Self-contained: Data fetch -> Preprocessing -> Training -> Evaluation -> Save resultsGenerated by builder_model agent"""

In [ ]:
# ============================================================# 1. IMPORTS

In [ ]:
# ============================================================import torchimport torch.nn as nnimport torch.optim as optimfrom torch.utils.data import DataLoader, TensorDatasetimport optunaimport pandas as pdimport numpy as npimport jsonimport osfrom datetime import datetimefrom scipy.stats import percentileofscore# Set random seeds for reproducibilitytorch.manual_seed(42)np.random.seed(42)if torch.cuda.is_available():    torch.cuda.manual_seed_all(42)# Device configuration (use GPU if available)device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')print(f"Using device: {device}")

In [ ]:
# ============================================================# 2. DATA FETCHING & PREPROCESSING

In [ ]:
# ============================================================def fetch_and_preprocess():    """    Self-contained data fetching and feature engineering.    No external file dependencies.    Returns: DataFrame with 8 engineered features    """    # === Install dependencies if needed ===    try:        from fredapi import Fred    except ImportError:        import subprocess        subprocess.run(["pip", "install", "fredapi"], check=True)        from fredapi import Fred    try:        import yfinance as yf    except ImportError:        import subprocess        subprocess.run(["pip", "install", "yfinance"], check=True)        import yfinance as yf    # === Load API key (Kaggle Secrets only, no fallback) ===    api_key = os.environ.get('FRED_API_KEY')    if api_key is None:        try:            from kaggle_secrets import UserSecretsClient            api_key = UserSecretsClient().get_secret("FRED_API_KEY")        except Exception as e:            raise RuntimeError(                "FRED_API_KEY not found in environment or Kaggle Secrets. "                "Please configure FRED_API_KEY in Kaggle Secrets."            ) from e    fred = Fred(api_key=api_key)    # === Fetch DFII10 from FRED ===    print("Fetching DFII10 from FRED...")    # Fetch from 2013-06-01 to allow 252-day rolling windows before schema start (2015-01-30)    series = fred.get_series('DFII10', observation_start='2013-06-01')    df = pd.DataFrame({'level': series})    df.index = pd.to_datetime(df.index)    df.index.name = 'Date'    # Drop NaN rows (holidays)    df = df.dropna()    print(f"FRED data shape after dropping NaN: {df.shape}")    # === Align with gold trading days ===    print("Fetching gold trading days from yfinance...")    gold = yf.download('GC=F', start='2013-06-01', progress=False)    gold_dates = pd.DatetimeIndex(gold.index)    # Reindex to gold trading days and forward-fill (max 5 days)    df = df.reindex(gold_dates)    df['level'] = df['level'].ffill(limit=5)    df = df.dropna()    print(f"Data shape after alignment to gold trading days: {df.shape}")    # === Feature Engineering ===    print("Computing 8 hand-crafted features...")    # Feature 2: Daily change    df['change_1d'] = df['level'].diff()    # Rolling std for normalization (60-day window)    rolling_std_60d = df['change_1d'].rolling(60, min_periods=60).std()    # Feature 3: Velocity 20-day (normalized)    df['velocity_20d'] = (df['level'] - df['level'].shift(20)) / rolling_std_60d    # Feature 4: Velocity 60-day (normalized)    df['velocity_60d'] = (df['level'] - df['level'].shift(60)) / rolling_std_60d    # Feature 5: Acceleration (change in 20-day velocity)    df['accel_20d'] = df['velocity_20d'] - df['velocity_20d'].shift(20)    # Feature 6: Rolling std 20-day    df['rolling_std_20d'] = df['change_1d'].rolling(20, min_periods=20).std()    # Feature 7: Regime percentile (252-day rolling percentile rank)    def percentile_rank(x):        if len(x) < 2:            return np.nan        return percentileofscore(x, x.iloc[-1]) / 100.0    df['regime_percentile'] = df['level'].rolling(252, min_periods=252).apply(percentile_rank, raw=False)    # Feature 8: Autocorrelation of daily changes (60-day window, lag 1)    def rolling_autocorr(x):        if len(x) < 2:            return np.nan        try:            return pd.Series(x).autocorr(lag=1)        except:            return np.nan    df['autocorr_20d'] = df['change_1d'].rolling(60, min_periods=60).apply(rolling_autocorr, raw=False)    # Drop rows with NaN from rolling calculations    initial_rows = len(df)    df = df.dropna()    print(f"Dropped {initial_rows - len(df)} rows due to rolling window NaN")    # === Align to schema date range ===    schema_start = '2015-01-30'    schema_end = '2025-02-12'    df = df.loc[schema_start:schema_end]    print(f"Final shape after schema alignment: {df.shape}")    print(f"Date range: {df.index.min()} to {df.index.max()}")    # Verify we have 8 feature columns    feature_cols = ['level', 'change_1d', 'velocity_20d', 'velocity_60d',                    'accel_20d', 'rolling_std_20d', 'regime_percentile', 'autocorr_20d']    assert all(col in df.columns for col in feature_cols), "Missing feature columns"    return df[feature_cols]

In [ ]:
# ============================================================# 3. DATASET CLASS

In [ ]:
# ============================================================class SlidingWindowDataset(torch.utils.data.Dataset):    """Dataset for sliding window autoencoder input"""    def __init__(self, data_array, window_size):        """        Args:            data_array: numpy array of shape (n_samples, n_features)            window_size: number of time steps in each window        """        self.data = torch.FloatTensor(data_array)        self.window_size = window_size    def __len__(self):        return len(self.data) - self.window_size + 1    def __getitem__(self, idx):        # Extract window and flatten        window = self.data[idx:idx + self.window_size]        return window.flatten()

In [ ]:
# ============================================================# 4. MODEL DEFINITION

In [ ]:
# ============================================================class RealRateAutoencoder(nn.Module):    """    MLP Autoencoder for real interest rate dynamics    Input: Flattened sliding window of shape (window_size * n_features,)    Latent: 4 dimensions (submodel output)    Output: Reconstruction of input    """    def __init__(self, window_size=60, n_features=8, hidden_dim=64,                 latent_dim=4, dropout=0.2):        super().__init__()        input_dim = window_size * n_features        self.encoder = nn.Sequential(            nn.Linear(input_dim, hidden_dim),            nn.BatchNorm1d(hidden_dim),            nn.ReLU(),            nn.Dropout(dropout),            nn.Linear(hidden_dim, hidden_dim // 2),            nn.BatchNorm1d(hidden_dim // 2),            nn.ReLU(),            nn.Dropout(dropout),            nn.Linear(hidden_dim // 2, latent_dim),            nn.Tanh()        )        self.decoder = nn.Sequential(            nn.Linear(latent_dim, hidden_dim // 2),            nn.BatchNorm1d(hidden_dim // 2),            nn.ReLU(),            nn.Dropout(dropout),            nn.Linear(hidden_dim // 2, hidden_dim),            nn.BatchNorm1d(hidden_dim),            nn.ReLU(),            nn.Dropout(dropout),            nn.Linear(hidden_dim, input_dim)        )        self.window_size = window_size        self.n_features = n_features        self.latent_dim = latent_dim    def encode(self, x):        """x: [batch, window_size * n_features] -> [batch, latent_dim]"""        return self.encoder(x)    def decode(self, z):        """z: [batch, latent_dim] -> [batch, window_size * n_features]"""        return self.decoder(z)    def forward(self, x):        """Full forward pass for training"""        z = self.encode(x)        reconstruction = self.decode(z)        return reconstruction, z    def transform(self, data_tensor):        """        Generate latent features from input windows        Args:            data_tensor: [batch, window_size * n_features]        Returns:            latent features: [batch, latent_dim]        """        self.eval()        with torch.no_grad():            _, z = self.forward(data_tensor)        return z

In [ ]:
# ============================================================# 5. TRAINING LOOP

In [ ]:
# ============================================================def train_model(model, train_loader, val_loader, config):    """    Train autoencoder with early stopping    Args:        model: RealRateAutoencoder instance        train_loader: DataLoader for training data        val_loader: DataLoader for validation data        config: dict with hyperparameters    Returns:        model: trained model        metrics: dict with training metrics    """    optimizer = optim.Adam(        model.parameters(),        lr=config['learning_rate'],        weight_decay=config.get('weight_decay', 1e-5)    )    scheduler = optim.lr_scheduler.ReduceLROnPlateau(        optimizer, mode='min', patience=5, factor=0.5    )    criterion = nn.MSELoss()    best_val_loss = float('inf')    best_train_loss = float('inf')    patience_counter = 0    best_state = None    max_epochs = config.get('max_epochs', 100)    patience = config.get('early_stop_patience', 10)    for epoch in range(max_epochs):        # === Training ===        model.train()        train_loss = 0.0        for batch in train_loader:            batch = batch.to(device)            optimizer.zero_grad()            reconstruction, _ = model(batch)            loss = criterion(reconstruction, batch)            loss.backward()            nn.utils.clip_grad_norm_(model.parameters(), 1.0)            optimizer.step()            train_loss += loss.item()        train_loss /= len(train_loader)        # === Validation ===        model.eval()        val_loss = 0.0        with torch.no_grad():            for batch in val_loader:                batch = batch.to(device)                reconstruction, _ = model(batch)                loss = criterion(reconstruction, batch)                val_loss += loss.item()        val_loss /= len(val_loader)        scheduler.step(val_loss)        # === Early stopping ===        if val_loss < best_val_loss:            best_val_loss = val_loss            best_train_loss = train_loss            patience_counter = 0            best_state = {k: v.clone().cpu() for k, v in model.state_dict().items()}        else:            patience_counter += 1            if patience_counter >= patience:                print(f"Early stopping at epoch {epoch + 1}")                break        if (epoch + 1) % 10 == 0 or epoch == 0:            print(f"Epoch {epoch + 1:3d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")    # Restore best weights    if best_state:        model.load_state_dict(best_state)    overfit_ratio = best_val_loss / (best_train_loss + 1e-10)    return model, {        'train_loss': best_train_loss,        'val_loss': best_val_loss,        'overfit_ratio': overfit_ratio,        'epochs_trained': epoch + 1    }

In [ ]:
# ============================================================# 6. OPTUNA HPO

In [ ]:
# ============================================================def run_hpo(train_data, val_data, n_trials, timeout):    """    Hyperparameter optimization using Optuna    Args:        train_data: numpy array of training features        val_data: numpy array of validation features        n_trials: number of Optuna trials        timeout: timeout in seconds    Returns:        best_params: dict of best hyperparameters        best_value: best validation loss        n_completed: number of completed trials    """    def objective(trial):        # === Search space from architect design ===        window_size = trial.suggest_categorical('window_size', [20, 40, 60])        hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])        dropout = trial.suggest_float('dropout', 0.1, 0.4)        learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)        weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)        config = {            'window_size': window_size,            'hidden_dim': hidden_dim,            'dropout': dropout,            'learning_rate': learning_rate,            'weight_decay': weight_decay,            'max_epochs': 100,            'early_stop_patience': 10,            'batch_size': 64        }        # Create model and move to device        model = RealRateAutoencoder(            window_size=window_size,            n_features=8,            hidden_dim=hidden_dim,            latent_dim=4,            dropout=dropout        ).to(device)        # Create datasets and loaders        train_dataset = SlidingWindowDataset(train_data, window_size)        val_dataset = SlidingWindowDataset(val_data, window_size)        train_loader = DataLoader(            train_dataset,            batch_size=config['batch_size'],            shuffle=True        )        val_loader = DataLoader(            val_dataset,            batch_size=config['batch_size'],            shuffle=False        )        # Train        _, metrics = train_model(model, train_loader, val_loader, config)        return metrics['val_loss']    study = optuna.create_study(        direction='minimize',        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)    )    study.optimize(objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True)    return study.best_params, study.best_value, len(study.trials)

In [ ]:
# ============================================================# 7. MAIN EXECUTION

In [ ]:
# ============================================================
# 7. MAIN EXECUTION
# ============================================================
if __name__ == "__main__":
    print("=" * 60)
    print("Gold SubModel Training: real_rate attempt 1")
    print("=" * 60)
    print(f"Started: {datetime.now().isoformat()}")

    # === 1. Data Fetching ===
    print("\n" + "=" * 60)
    print("STEP 1: Data Fetching & Feature Engineering")
    print("=" * 60)
    full_data = fetch_and_preprocess()
    
    print(f"\nFull data shape: {full_data.shape}")
    print(f"Feature columns: {list(full_data.columns)}")
    print(f"\nFeature statistics:")
    print(full_data.describe())

    # Store schema range for validation
    schema_start = '2015-01-30'
    schema_end = '2025-02-12'

    # === 2. Train/Val/Test Split (70/15/15, chronological) ===
    print("\n" + "=" * 60)
    print("STEP 2: Data Splitting")
    print("=" * 60)
    
    n_total = len(full_data)
    n_train = int(0.7 * n_total)
    n_val = int(0.15 * n_total)
    
    train_data = full_data.iloc[:n_train].values
    val_data = full_data.iloc[n_train:n_train + n_val].values
    test_data = full_data.iloc[n_train + n_val:].values
    
    print(f"Train samples: {len(train_data)}")
    print(f"Val samples:   {len(val_data)}")
    print(f"Test samples:  {len(test_data)}")

    # === 3. Standardization (fit on train only) ===
    print("\n" + "=" * 60)
    print("STEP 3: Standardization")
    print("=" * 60)
    
    train_mean = train_data.mean(axis=0)
    train_std = train_data.std(axis=0)
    
    train_data = (train_data - train_mean) / (train_std + 1e-8)
    val_data = (val_data - train_mean) / (train_std + 1e-8)
    test_data = (test_data - train_mean) / (train_std + 1e-8)
    
    print(f"Train mean: {train_mean}")
    print(f"Train std: {train_std}")

    # === 4. Hyperparameter Optimization ===
    print("\n" + "=" * 60)
    print("STEP 4: Optuna Hyperparameter Optimization")
    print("=" * 60)
    
    best_params, best_value, n_completed = run_hpo(
        train_data, val_data,
        n_trials=5,      # Smoke test: 5 trials
        timeout=600       # 10 minutes max
    )
    
    print(f"\nOptuna completed {n_completed} trials")
    print(f"Best validation loss: {best_value:.6f}")
    print(f"Best hyperparameters:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")

    # === 5. Final Training with Best Params ===
    print("\n" + "=" * 60)
    print("STEP 5: Final Training with Best Parameters")
    print("=" * 60)
    
    final_config = {
        **best_params,
        'max_epochs': 100,
        'early_stop_patience': 10,
        'batch_size': 64
    }
    
    model = RealRateAutoencoder(
        window_size=best_params['window_size'],
        n_features=8,
        hidden_dim=best_params['hidden_dim'],
        latent_dim=4,
        dropout=best_params['dropout']
    ).to(device)
    
    window_size = best_params['window_size']
    
    train_dataset = SlidingWindowDataset(train_data, window_size)
    val_dataset = SlidingWindowDataset(val_data, window_size)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    
    model, metrics = train_model(model, train_loader, val_loader, final_config)
    
    print(f"\nFinal model metrics:")
    print(f"  Train Loss: {metrics['train_loss']:.6f}")
    print(f"  Val Loss: {metrics['val_loss']:.6f}")
    print(f"  Overfit Ratio: {metrics['overfit_ratio']:.3f}")
    print(f"  Epochs Trained: {metrics['epochs_trained']}")

    # === 6. Generate Submodel Output ===
    print("\n" + "=" * 60)
    print("STEP 6: Generating Latent Features")
    print("=" * 60)
    
    # Standardize full data for inference
    full_data_std = (full_data.values - train_mean) / (train_std + 1e-8)
    
    # Create dataset for full data
    full_dataset = SlidingWindowDataset(full_data_std, window_size)
    full_loader = DataLoader(full_dataset, batch_size=64, shuffle=False)
    
    # Generate latent features
    latent_features = []
    model.eval()
    with torch.no_grad():
        for batch in full_loader:
            batch = batch.to(device)
            z = model.transform(batch)
            latent_features.append(z.cpu().numpy())

    latent_features = np.vstack(latent_features)

    # Align with dates (first window_size-1 rows will be NaN)
    output_df = pd.DataFrame(
        index=full_data.index,
        columns=['real_rate_latent_0', 'real_rate_latent_1',
                 'real_rate_latent_2', 'real_rate_latent_3']
    )

    # Fill in latent features (starting from index window_size-1)
    output_df.iloc[window_size - 1:] = latent_features

    print(f"Output shape before dropping NaN: {output_df.shape}")
    print(f"NaN rows (due to windowing): {output_df.isna().any(axis=1).sum()}")

    # Drop NaN rows to ensure clean output
    output_df = output_df.dropna()

    print(f"Output shape after dropping NaN: {output_df.shape}")
    print(f"Date range: {output_df.index.min()} to {output_df.index.max()}")
    print(f"\nFirst few rows:")
    print(output_df.head())
    print(f"\nLast few rows:")
    print(output_df.tail())
    print(f"\nLatent feature statistics:")
    print(output_df.describe())

    # Validate schema alignment
    schema_start_dt = pd.Timestamp(schema_start)
    schema_end_dt = pd.Timestamp(schema_end)

    if output_df.index.min() > schema_start_dt:
        print(f"\nWARNING: Output starts at {output_df.index.min()}, but schema requires {schema_start_dt}")
        print("This means insufficient data buffer for the window size.")
        raise ValueError(f"Output date range does not cover schema start date. Need more historical data.")

    if output_df.index.max() < schema_end_dt:
        print(f"\nWARNING: Output ends at {output_df.index.max()}, but schema requires {schema_end_dt}")
        raise ValueError(f"Output date range does not cover schema end date.")

    print(f"\n✓ Schema validation passed: output covers {schema_start} to {schema_end}")

    # === 7. Save Results ===
    print("\n" + "=" * 60)
    print("STEP 7: Saving Results")
    print("=" * 60)

    # Save submodel output CSV
    output_df.to_csv("submodel_output.csv")
    print("Saved: submodel_output.csv")

    # Save model state
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': final_config,
        'train_mean': train_mean,
        'train_std': train_std
    }, "model.pt")
    print("Saved: model.pt")

    # Save training result JSON
    result = {
        "feature": "real_rate",
        "attempt": 1,
        "timestamp": datetime.now().isoformat(),
        "best_params": best_params,
        "metrics": {
            "train_loss": float(metrics['train_loss']),
            "val_loss": float(metrics['val_loss']),
            "overfit_ratio": float(metrics['overfit_ratio']),
            "epochs_trained": int(metrics['epochs_trained'])
        },
        "optuna_trials_completed": n_completed,
        "optuna_best_value": float(best_value),
        "output_shape": list(output_df.shape),
        "output_columns": list(output_df.columns),
        "data_info": {
            "train_samples": len(train_data),
            "val_samples": len(val_data),
            "test_samples": len(test_data),
            "full_samples": len(full_data),
            "window_size": window_size,
            "n_features": 8,
            "latent_dim": 4
        },
        "standardization": {
            "train_mean": train_mean.tolist(),
            "train_std": train_std.tolist()
        }
    }

    with open("training_result.json", "w") as f:
        json.dump(result, f, indent=2)
    print("Saved: training_result.json")

    print("\n" + "=" * 60)
    print("Training Complete!")
    print("=" * 60)
    print(f"Finished: {datetime.now().isoformat()}")
    
    print("\nOutput files:")
    print("  - submodel_output.csv (latent features)")
    print("  - model.pt (model weights + config)")
    print("  - training_result.json (metrics + metadata)")